# Train data preparation

In [1]:
# Librariesss
import pandas as pd
import xmltodict, json
import numpy as np
import xml.etree.ElementTree as ET

In [16]:
# We get the data from link, because of laziness
df = pd.read_csv('https://trein.fwrite.org/AMS-Aurora-archive/2016-01/DVS_2016-01-01.csv.xz')
df.columns = ["date", "data", "code"]

In [59]:
# Defining RitStation class here
class RitStation:
    def __init__(self, StationCode,KorteNaam, LangeNaam):
        self.StationCode = StationCode
        self.KorteNaam = KorteNaam
        self.LangeNaam = LangeNaam

# Defining Train class here
class Trein:
    def __init__(self, TreinNumber, TreinSoort, Vervoerder, TreinStation_Code,TreinStation_Name, VertekTijd, ExacteVertrekVertraging):
        self.TreinNumber = TreinNumber
        self.TreinSoort = TreinSoort
        self.Vervoerder = Vervoerder
        self.TreinStation_Code = TreinStation_Code
        self.TreinStation_Name = TreinStation_Name
        self.VertekTijd = VertekTijd
        self.ExacteVertrekVertraging = ExacteVertrekVertraging

In [64]:
# Getting only 30 rows
smalldf = df.head(30)

# The xml data
xml_data = smalldf["data"]

# print(xml_data[2])
# print(ET.fromstring(xml_data[1])[0])
# Lists to store the train and ritstation objects
ritstations = []
trains = []

# It goes through every row in the data (1 day)
for i,train_data in enumerate(xml_data):
    root = ET.fromstring(train_data)
    
    # Check if there is always root
    if len(root) != 0:
        
        # Gets the elements manually
        # Ritstation contain it's properties, same for train
        wrapper = root[0][1]
        ritstation = wrapper[2]
        train = wrapper[3]
        # Planned and actual stations are part of the train parent
        planned_station = train[11]
        actual_station = train[12]

        # Gets the properties of the ritstation manually
        statcode = ritstation[0].text
        kortenaam = ritstation[2].text
        langenaam = ritstation[4].text

        # Here the ritstation object is created with the data above
        station = RitStation(statcode,kortenaam,langenaam)
        ritstations.append(station)

        # Gets the properties of the train manually
        trainnumber = train[0].text
        trainsoort = train[1].text
        vervoerder = train[4].text
        stationCode = actual_station[0].text
        stationName = actual_station[2].text
        vertektijd_planned = train[13].text
        vertekijd_actual = train[14].text
        ExacteVertrekVertraging = train[15].text

        # Here the ritstation object is created with the data above
        train = Trein(trainnumber,trainsoort,vervoerder, stationCode, stationName, vertektijd_planned,ExacteVertrekVertraging)
        trains.append(train)

In [65]:
# Dataframes are created to visualize the object
df_stations = pd.DataFrame()
df_trains = pd.DataFrame()

# It goes through the objects of the ritstations and trains lists, storing them into the dataframe
for stat in ritstations:
    sss = pd.DataFrame(stat.__dict__.values(),station.__dict__.keys()).T
    df_stations = df_stations.append(sss,ignore_index=True)
for train_data in trains:
    ttt = pd.DataFrame(train_data.__dict__.values(), train_data.__dict__.keys()).T
    df_trains = df_trains.append(ttt, ignore_index=True)

# Here the two dataframes are combined into one
df_combined = pd.concat(
    [df_stations,df_trains],
    axis=1,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)

# Yay
df_combined

,StationCode,KorteNaam,LangeNaam,TreinNumber,TreinSoort,Vervoerder,TreinStation_Code,TreinStation_Name,VertekTijd,ExacteVertrekVertraging
0,HLMS,Spaarnwde,Haarlem Spaarnwoude,14882,Sprinter,NS,HLMS,Spaarnwde,2016-01-01T00:10:00Z,PT0S
1,NSCH,Nweschans,Bad Nieuweschans,37598,Stoptrein,Arriva,NSCH,Nweschans,2016-01-01T00:10:00Z,PT0S
2,DVD,Duivendt,Duivendrecht,7495,Sprinter,NS,DVD,Duivendt,2016-01-01T00:11:00Z,PT0S
3,DDRS,Stadspldrs,Dordrecht Stadspolders,36795,Stoptrein,Arriva,DDRS,Stadspldrs,2016-01-01T00:11:00Z,PT0S
4,EMNZ,Emmen Z,Emmen Zuid,8083,Stoptrein,Arriva,EMNZ,Emmen Z,2016-01-01T00:11:00Z,PT0S
5,BNZ,Barnevld Z,Barneveld Zuid,31378,Stoptrein,Valleilijn,BNZ,Barnevld Z,2016-01-01T00:11:00Z,PT0S
6,LWC,Camminghab,Leeuwarden Camminghaburen,37496,Stoptrein,Arriva,LWC,Camminghab,2016-01-01T00:12:00Z,PT0S
7,RTB,Blaak,Rotterdam Blaak,15188,Sprinter,NS,RTB,Blaak,2016-01-01T00:12:00Z,PT0S
8,KLP,De Klomp,Veenendaal-De Klomp,3091,Intercity,NS,KLP,De Klomp,2016-01-01T00:12:00Z,PT0S
9,GND,Hardinxvld,Hardinxveld-Giessendam,36788,Stoptrein,Arriva,GND,Hardinxvld,2016-01-01T00:12:00Z,PT0S
